In [ ]:
from jinja2 import Template
import datetime
import pandas as pd

In [ ]:
import json

In [ ]:
# Your dynamic data
with open("youtube-wrapped.json") as f:
    data = json.loads(f.read())    

In [ ]:
from datetime import datetime

def format_human_date(dt: datetime) -> str:
    # Suffix helper
    def ordinal(n):
        if 11 <= n % 100 <= 13:
            suffix = 'th'
        else:
            suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
        return f"{n}{suffix}"

    return f"{dt.strftime('%A')} the {ordinal(dt.day)} of {dt.strftime('%B')}"


In [ ]:
# Prepare processed fields
data["top_channels_combined"] = list(zip(data["top_channels"], data["top_channels_links"]))
data["top_videos_combined"] = list(zip(data["top_videos"], data["top_videos_thumbs"], data["top_videos_links"]))

top_date_dt = pd.to_datetime(f"{data['top_day_date_day']:02d}-{data['top_day_date_month']:02d}-{data['top_day_date_year']}", dayfirst=True)

data["top_day_date"] = format_human_date(top_date_dt)

with open("youtube-wrapped-template.html.tmpl", "r", encoding="utf-8") as f:
    template_html = f.read()

template = Template(template_html)

# Render the final HTML
rendered_html = template.render(
    year=data["year"],
    total_hours=data["total_hours"],
    total_minutes=data["total_minutes"],
    average_hours=data["average_hours"],
    average_minutes=data["average_minutes"],
    top_day_date=data["top_day_date"],
    top_day_minutes=data["top_day_minutes"],
    top_channels=data["top_channels_combined"],
    top_videos=data["top_videos_combined"],
    category_names=data["top_categories"],
    total_days=data["total_days"],
)

# Write to output file
with open('youtube-wrapped.html', 'w', encoding='utf-8') as f:
    f.write(rendered_html)

print("Page written to youtube-wrapped.html ✅")